1. Which Nationalities has the maximum number of players - Done
2. Club with most no. of loaned players. - Done
3. Clubs with most spendings on players - Done
4. Is wage of a player related to club -Done
5. Is wage of a player related to Age - Done
6. Does club choose player based on overall score or International reputaion - Done

In [ ]:
#Importing necessary files

import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
# Standard plotly imports
import chart_studio.plotly as py
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode
# Using plotly + cufflinks in offline mode
import cufflinks
cufflinks.go_offline(connected=True)
init_notebook_mode(connected=True)

import plotly.offline as py

init_notebook_mode(connected = True)
import plotly.graph_objs as go


%matplotlib inline

In [ ]:
#importing file
fifa = pd.read_csv("../input/fifa19/data.csv", usecols=['ID','Name','Age', 'Nationality', 'Overall', 'Club', 'Value','Wage', 'Preferred Foot', 'International Reputation', 'Loaned From', 'Height', 'Weight'])

fifa.head()

In [ ]:
#Converting wages to numeric value

fifa["Wage"] = fifa["Wage"].apply(lambda x: x.replace("€",""))
fifa["Wage"] = fifa["Wage"].apply(lambda x: x.replace("K","000"))

fifa = fifa.astype({'Wage': 'int64'})


In [ ]:
fifa.info()

### Which Nationalities has the maximum number of players

In [ ]:
rate = pd.DataFrame(fifa.groupby(['Nationality'])['ID'].count().reset_index())
country_list = list(rate["Nationality"])
player_countrywise = list(rate["ID"])

data = dict (
    type = 'choropleth',
    locations = country_list,
    locationmode='country names',
    colorscale = ['magma_r'],
    z=player_countrywise)

In [ ]:
map = go.Figure(data=[data])
py.offline.iplot(map)

### Club with most no. of loaned players.

In [ ]:
loaned_player_data = fifa.dropna(subset=['Loaned From']).copy()
loaned_player = loaned_player_data.groupby(by='Club').count()['Loaned From'].sort_values(ascending=False)
loaned_player.head(5)

In [ ]:
loaned_player.iloc[0:10].plot.bar(x='index')

### Clubs with most spendings on players

In [ ]:
total_wages = pd.DataFrame(fifa.groupby("Club")["Wage"].sum().sort_values(ascending = False))
total = int(total_wages[:10].sum())

In [ ]:
wage_list = list(total_wages["Wage"])
size = len(wage_list)-1
count = 0
while total>0:
    total -= wage_list[size-count]
    count += 1
    
count

In [ ]:
top_wages = pd.DataFrame(fifa.groupby("Club")["Wage"].sum().sort_values(ascending = False).head(10))
plt.figure(figsize=(18,8))
plt.barh(list(top_wages.index), list(top_wages["Wage"]))

plt.ticklabel_format(style='plain', axis='x')


for index, value in enumerate(list(top_wages["Wage"])):
    plt.text(value, index, str(value))
    
plt.legend()
plt.show()

### Is wage of a player related to club

In [ ]:
#Droping na values for club
fifa.isnull().sum()
fifa.dropna(subset=['Club'], inplace= True)

In [ ]:
mean_wage = pd.DataFrame(fifa.groupby("Club")["Wage"].mean().sort_values(ascending = False).head(10)).sort_index()

In [ ]:
club_list = list(mean_wage.index)

In [ ]:
df_std = fifa[["Club","Wage"]].copy()

#df_std = pd.DataFrame(df_std["Club"].apply(lambda x: x if x in club_list else np.nan))
df_std = df_std[df_std.Club.str.contains('|'.join(club_list))]
#df_std.dropna(inplace = True)
df_std

In [ ]:
std_wage = pd.DataFrame(df_std.groupby("Club")["Wage"].std().sort_values(ascending = False).head(10)).sort_index()

In [ ]:
plt.bar(list(mean_wage.index),list(mean_wage["Wage"]), label = "Mean")
plt.bar(list(std_wage.index),list(std_wage["Wage"]), label = "Std")
plt.xticks(rotation=90) 
plt.legend()
plt.show()

In [ ]:
min_wage = pd.DataFrame(df_std.groupby("Club")["Wage"].min().sort_values(ascending = False)).sort_index()
min_wage

In [ ]:
max_wage = pd.DataFrame(df_std.groupby("Club")["Wage"].max().sort_values(ascending = False)).sort_index()
max_wage

In [ ]:
animals=['giraffes', 'orangutans', 'monkeys']
max_wage_list = list(max_wage["Wage"])
min_wage_list = list(min_wage["Wage"])

fig = go.Figure(data=[
    go.Bar(name='Max', x=list(max_wage.index), y=max_wage_list,text=max_wage_list,textposition='auto'),
    go.Bar(name='Min', x=list(min_wage.index), y=min_wage_list,text=min_wage_list,textposition='auto')
])
# Change the bar mode
fig.update_layout(barmode='group')
fig.show()

### Is wage of a player related to Age

In [ ]:
age_dataframe_mean = pd.DataFrame(fifa.groupby("Age")["Wage"].mean())
age_dataframe_mean.rename(columns = {'Wage':'Avg_Wage'}, inplace=True)

In [ ]:
age_dataframe_std = pd.DataFrame(fifa.groupby("Age")["Wage"].std())
age_dataframe_std.rename(columns = {'Wage':'Std_Wage'}, inplace=True)

In [ ]:
plt.plot(list(age_dataframe_mean.index), list(age_dataframe_mean["Avg_Wage"]))
plt.plot(list(age_dataframe_std.index), list(age_dataframe_std["Std_Wage"]))

plt.xticks() 
plt.legend()
plt.show()

### Does club choose player based on overall score or International reputaion

In [ ]:
corr = fifa[["Wage","International Reputation", "Overall"]].corr(method ='pearson')
sns.heatmap(corr, 
            xticklabels=corr.columns.values,
            yticklabels=corr.columns.values, 
            annot=True,
           cmap="YlOrRd")